Helper files from: 
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models


AlexNet model settings and training from:
- Github specific file: https://github.com/l5shi/Image-Recognition-on-MNIST-dataset/blob/master/AlexNet.ipynb

In [1]:
import os
import time
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms
from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
True


In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 256
NUM_EPOCHS = 40

# Architecture
NUM_CLASSES = 10

# Other
dtype = torch.float
device = torch.device("mps")
set_all_seeds(RANDOM_SEED)

# Deterministic behavior not yet supported by AdaptiveAvgPool2d
#set_deterministic()

import sys

sys.path.insert(0, "..") # to include ../helper_evaluate.py etc.

from helper_evaluate import compute_accuracy
from helper_data import get_dataloaders_mnist
from helper_train import train_classifier_simple_v1

### Set random seed ###
set_all_seeds(RANDOM_SEED)

In [3]:
##########################
### Dataset
##########################

train_transforms = transforms.Compose([transforms.RandomCrop((28, 28)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.CenterCrop((28, 28)),
                                      transforms.ToTensor()])


train_loader, valid_loader, test_loader = get_dataloaders_mnist (
    batch_size=BATCH_SIZE, 
    num_workers=2, 
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    validation_fraction=0.1)

# Checking the dataset
print('Training Set:\n')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break
    
# Checking the dataset
print('\nValidation Set:')
for images, labels in valid_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

# Checking the dataset
print('\nTesting Set:')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

Training Set:

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([8, 3, 7, 7, 4, 7, 5, 4, 3, 7])

Validation Set:
Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([0, 3, 7, 6, 1, 4, 2, 2, 0, 9])

Testing Set:
Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([9, 3, 0, 3, 4, 2, 5, 1, 6, 5])


In [4]:
class AlexNet(nn.Module):   
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),   
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),
         
        )
    
    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x

In [5]:
torch.manual_seed(RANDOM_SEED)

model = AlexNet()
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             832
              ReLU-2           [-1, 32, 26, 26]               0
            Conv2d-3           [-1, 64, 26, 26]          18,496
              ReLU-4           [-1, 64, 26, 26]               0
         MaxPool2d-5           [-1, 64, 13, 13]               0
            Conv2d-6           [-1, 96, 13, 13]          55,392
              ReLU-7           [-1, 96, 13, 13]               0
            Conv2d-8           [-1, 64, 13, 13]          55,360
              ReLU-9           [-1, 64, 13, 13]               0
           Conv2d-10           [-1, 32, 13, 13]          18,464
             ReLU-11           [-1, 32, 13, 13]               0
        MaxPool2d-12           [-1, 32, 12, 12]               0
           Linear-13                 [-1, 2048]       9,439,232
             ReLU-14                 [-

In [11]:

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [12]:
log_dict = train_classifier_simple_v1(num_epochs=NUM_EPOCHS, model=model, 
                                      optimizer=optimizer, device=device, 
                                      train_loader=train_loader, valid_loader=valid_loader, 
                                      logging_interval=25)

Epoch: 001/040 | Batch 0000/0210 | Loss: 2.3029


KeyboardInterrupt: 

In [ ]:
with torch.set_grad_enabled(False):
    
    train_acc = compute_accuracy(model=model,
                                 data_loader=test_loader,
                                 device=device)
    
    test_acc = compute_accuracy(model=model,
                                data_loader=test_loader,
                                device=device)
    
    valid_acc = compute_accuracy(model=model,
                                 data_loader=valid_loader,
                                 device=device)
    

print(f'Train ACC: {valid_acc:.2f}%')
print(f'Validation ACC: {valid_acc:.2f}%')
print(f'Test ACC: {test_acc:.2f}%')

# EXTRACT WEIGHTS

In [13]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [14]:
path = "../../files/preload/MNIST/AlexNet/"

In [15]:
for i in params:
    name = i[0].split(".")
    for ii in range(3):
        file_name = name[2]+str(int(name[1])+1)+"_"+str(ii)
        np.savetxt(fname=f"{path}/{file_name}", delimiter=" ", X=i[1].flatten())

weight1_0
weight1_1
weight1_2
bias1_0
bias1_1
bias1_2
weight3_0
weight3_1
weight3_2
bias3_0
bias3_1
bias3_2
weight6_0
weight6_1
weight6_2
bias6_0
bias6_1
bias6_2
weight8_0
weight8_1
weight8_2
bias8_0
bias8_1
bias8_2
weight10_0
weight10_1
weight10_2
bias10_0
bias10_1
bias10_2
weight2_0
weight2_1
weight2_2
bias2_0
bias2_1
bias2_2
weight5_0
weight5_1
weight5_2
bias5_0
bias5_1
bias5_2
weight7_0
weight7_1
weight7_2
bias7_0
bias7_1
bias7_2


In [ ]:

# Write the used inputs and outputs
#np.savetxt(fname=f"{path}/input_0", delimiter=" ", X=images.cuda().view(-1, 3 * 33 * 33).tolist())
#np.savetxt(fname=f"{path}/outputlayer8_0", delimiter=" ", X=model.output(images.cuda().view(-1, 3, 33, 33)).data.cpu().view(-1))